In [1]:
using Distributions 

In [ ]:
for t in 1:T 
    C = C_0
    if t > t_0
        X_bar_new = ((t-1)*X_bar_old + X_new)/t
        C_new = (t-1)/t*C_old + s/t*(t*X_bar_old*X_bar_old' - (t+1)*X_bar_new*X_bar_new' + X_new*X_new' + ε*eye(d))
    
        

In [ ]:
def ll(A, y, ξ, n_trials) :
    return y.dot(A.dot(ξ)) - n_trials.dot(np.log(1+np.exp(A.dot(ξ)))) 


def logistic_regression(X, y, T, n_trials, prior_mean, prior_sd, ξ_0, jump_σ, store_frequency=10) :
    d, N = np.shape(X)
    A = csc_matrix(X).transpose()
    ξ_chain = np.zeros((int(T/store_frequency),d))
    ξ_current = np.copy(ξ_0)
    ll_current = ll(A, y, ξ_current, n_trials)
    log_prior_current = (-1/2)*np.sum((ξ_current-prior_mean)**2/prior_sd**2)
    accepted = 0
    
    s = 2.4**2/d
    ɛ = 0.01
    i = 0

    for t in range(1,T+1) :
#         if t > 100 :
#             C = s*np.cov(ξ_chain[t-100:t].transpose()) + s*ɛ*np.identity(d)
#         else : 
#             C = jump_σ**2*np.identity(d) 
#         ξ_proposed = npr.multivariate_normal(mean=ξ_chain[t-1], cov=C, size=1).reshape(d,)
        ξ_proposed = ξ_current + jump_σ*npr.randn(d)
        ll_proposed = ll(A, y, ξ_proposed, n_trials)
        log_prior_proposed = (-1/2)*np.sum((ξ_proposed-prior_mean)**2/prior_sd**2)
        α = np.exp(ll_proposed + log_prior_proposed - ll_current - log_prior_current)
        
        if npr.uniform(size=1)[0] < α :
            ξ_current = np.copy(ξ_proposed)
            ll_current = np.copy(ll_proposed)
            log_prior_current = np.copy(log_prior_proposed)
            accepted += 1 
        if t%store_frequency == 0 : 
            ξ_chain[i] = np.copy(ξ_current)
            i += 1 
            
    return ξ_chain, accepted